In [ ]:
# This script creates a training set for a k-NN classifier for pose classification
# using MediaPipe Pose, exports it to a CSV, and prepares it for the ML Kit sample app.

# Import necessary libraries
import csv
import cv2
import numpy as np
import os
import tqdm

from mediapipe.python.solutions import drawing_utils as mp_drawing
from mediapipe.python.solutions   
import pose as mp_pose   



# Define folder paths
images_in_folder = 'fitness_poses_images_in'  # Folder containing pose images
images_out_folder = 'fitness_poses_images_out_basic'  # Output for processed images
csv_out_path = 'fitness_poses_csvs_out_basic.csv'  # Output CSV path

# Function to extract landmarks from an image
def extract_landmarks(image):
    """
    Extracts landmarks from an image using MediaPipe Pose.

    Args:
        image: The input image.

    Returns:
        A list of landmark coordinates, or None if no landmarks are detected.
    """

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        if results.pose_landmarks:   

            landmarks = [(landmark.x, landmark.y, landmark.z)
                          for landmark in results.pose_landmarks.landmark]
            return   
 landmarks
        else:
            return None


# Function to process a folder of images and create training data
def create_training_data(images_in_folder, images_out_folder, csv_out_path):
    """
    Processes images in a folder, extracts landmarks, and saves data to a CSV.

    Args:
        images_in_folder: Path to the folder containing pose images.
        images_out_folder: Path to save processed images with predicted pose.
        csv_out_path: Path to save the training data CSV.
    """

    # Create output folders if they don't exist
    os.makedirs(images_out_folder, exist_ok=True)

    # Open CSV file for writing
    with open(csv_out_path, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)

        # Iterate through each class subfolder in the input folder
        for class_name in os.listdir(images_in_folder):
            class_path = os.path.join(images_in_folder, class_name)

            # Iterate through each image in the class subfolder
            for image_name in tqdm.tqdm(os.listdir(class_path)):
                image_path = os.path.join(class_path, image_name)

                # Load image
                image = cv2.imread(image_path)

                # Extract landmarks
                landmarks = extract_landmarks(image.copy())

                if landmarks:
                    # Write data to CSV (class name, landmark coordinates)
                    writer.writerow([class_name] + [str(landmark[0]) + ',' + str(landmark[1]) + ',' + str(landmark[2]) for landmark in landmarks])

                    # Optionally save processed image with predicted pose (for verification)
                    # mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
                    # cv2.imwrite(os.path.join(images_out_folder, image_name), image)


# Process images and create training data
create_training_data(images_in_folder, images_out_folder, csv_out_path)

print(f"Training data saved to: {csv_out_path}")

# Note: You can manually review the processed images in images_out_folder
# to remove any incorrectly classified poses before using the CSV in the ML Kit app.